In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from collections import Counter
from imblearn.over_sampling import SMOTE
from datetime import datetime
from sklearn.decomposition import PCA

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# os.chdir('/content/drive/MyDrive/AI SGS Competition')

In [5]:
path = "Data/train"
os.listdir(path)
df1 = pd.DataFrame()
for file in os.listdir(path):
    if (file.endswith('.csv')) & (file.startswith('19')):
        df = pd.read_csv(path + "/"+ file)

        df.drop(['Filament'], axis=1, inplace=True) # Drop Filament column
        df.drop(['Emission'], axis=1, inplace=True) # Drop Emission column
        df.drop(['Temp'], axis=1, inplace=True) # Drop Temp column
        df.drop(['Test ID'], axis=1, inplace=True) # Drop Test ID column

        df['502_to_69_ratio'] = df['502 m/z'] / df['69 m/z'] # Create new column with ratio of 502 m/z to 69 m/z

        ############################################
        #clean the datetime format
        def clean_date(date):
            if 'UTC' in date:
                date = date.split('(')[0]
                date = date.split("  ")[0]
            return date

        def convert_to_datetime(date):
            date_string = date
            date_format = '%d/%m/%Y %I:%M %p'

            datetime_obj = datetime.strptime(date_string, date_format)
            formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
            return formatted_datetime

        df['Tune Date'] = df['Tune Date'].apply(lambda x: clean_date(x))
        if 'UTC' in df['Tune Date'][0]:
            df['Tune Date'] = df['Tune Date'].apply(lambda x: convert_to_datetime(x))

        df['Tune Date'] = pd.to_datetime(df['Tune Date'])
        df['total_hour_diff'] = df['Tune Date'].diff().apply(lambda x: x.total_seconds()/3600) # new column with total hour difference

        df.drop(['Tune Date'], axis=1, inplace=True) # Drop Tune Date column
        ############################################

        # Perform PCA on the highly correlated columns
        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['69 m/z', '70 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 69 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['219 m/z', '220 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 219 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['502 m/z', '503 m/z']])
        # print(pca.explained_variance_ratio_)
        df['PCA_Component 502 m/z'] = components

        # add columns for pct change
        def precent_change(col_name , periods):
            df[col_name + " pct change " + str(periods) + 'days'] = df[col_name].pct_change(periods=periods) * 100
            return df[col_name + " pct change " + str(periods) + 'days']

        for i in range (10):
            precent_change('EM Volts', i+1)
        for i in range (10):
            precent_change('502_to_69_ratio', i+1)

        df1 = pd.concat([df1,df])

df1.fillna(0, inplace=True) # fill all NaN values with 0

# # Perform one-hot encoding
# one_hot_encoded = pd.get_dummies(df1['System'], prefix='System')
# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# df1 = pd.concat([df1, one_hot_encoded], axis=1)
#drop System column
df1.drop(['System'], axis=1, inplace=True)
df1.drop(['ID'], axis=1, inplace=True)
df1

,69 m/z,70 m/z,219 m/z,220 m/z,502 m/z,503 m/z,Amu gain,Amu offset,EM Volts,Ent Lens,...,502_to_69_ratio pct change 1days,502_to_69_ratio pct change 2days,502_to_69_ratio pct change 3days,502_to_69_ratio pct change 4days,502_to_69_ratio pct change 5days,502_to_69_ratio pct change 6days,502_to_69_ratio pct change 7days,502_to_69_ratio pct change 8days,502_to_69_ratio pct change 9days,502_to_69_ratio pct change 10days
0,389888,4815,352320,15620,24096,2770,2503,127,1718,7.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,322880,3532,359616,15261,27592,2709,2496,126,1812,12.0,...,38.272862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,316608,3490,379840,16608,29992,3163,2491,126,1718,12.0,...,10.851483,53.277517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,296128,3154,368064,15829,28832,2954,2491,125,1765,12.0,...,2.780743,13.933977,57.539771,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,300544,2958,375296,16720,29136,2995,2490,125,1812,12.0,...,-0.430444,2.338329,13.443555,56.861651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,440512,5046,449856,19000,59912,5996,2478,127,1706,12.0,...,35.227228,14.444050,15.234680,13.936579,22.482853,-13.188058,-23.393296,-26.061671,-18.294253,-26.566907
15,449920,5018,409472,18504,53848,5196,2480,127,1659,14.5,...,-12.000905,18.998737,0.709728,1.405475,0.263158,7.783801,-23.606277,-32.586794,-34.934940,-28.099682
16,381056,4220,242816,9745,28904,2786,2485,128,1659,14.5,...,-36.622535,-44.228404,-24.581617,-36.172727,-35.731780,-36.455752,-31.689359,-51.583595,-57.275219,-58.763414
17,358720,3942,226816,10157,31592,3328,2480,128,1706,12.0,...,16.105391,-26.415346,-35.246171,-12.435191,-25.893095,-25.381132,-26.221702,-20.687663,-43.785943,-50.394226


In [6]:
path = "Data/test"
val = pd.DataFrame()
for file in os.listdir(path):
    if (file.endswith('.csv')) & (file.startswith('19')):
        df = pd.DataFrame()
        df = pd.read_csv(path + "/"+ file)

        df.drop(['Filament'], axis=1, inplace=True) # Drop Filament column
        df.drop(['Emission'], axis=1, inplace=True) # Drop Emission column
        df.drop(['Temp'], axis=1, inplace=True) # Drop Temp column
        df.drop(['Test ID'], axis=1, inplace=True) # Drop Test ID column

        df['502_to_69_ratio'] = df['502 m/z'] / df['69 m/z'] # Create new column with ratio of 502 m/z to 69 m/z

        ############################################
        #clean the datetime format
        def clean_date(date):
            if 'UTC' in date:
                date = date.split('(')[0]
                date = date.split("  ")[0]
            return date

        def convert_to_datetime(date):
            date_string = date
            date_format = '%d/%m/%Y %I:%M %p'

            datetime_obj = datetime.strptime(date_string, date_format)
            formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
            return formatted_datetime

        df['Tune Date'] = df['Tune Date'].apply(lambda x: clean_date(x))
        if 'UTC' in df['Tune Date'][0]:
            df['Tune Date'] = df['Tune Date'].apply(lambda x: convert_to_datetime(x))

        df['Tune Date'] = pd.to_datetime(df['Tune Date'])
        df['total_hour_diff'] = df['Tune Date'].diff().apply(lambda x: x.total_seconds()/3600) # new column with total hour difference

        df.drop(['Tune Date'], axis=1, inplace=True) # Drop Tune Date column
        ############################################

        # Perform PCA on the highly correlated columns
        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['69 m/z', '70 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 69 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['219 m/z', '220 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 219 m/z'] = components

        pca = PCA(n_components=1)
        components = pca.fit_transform(df[['502 m/z', '503 m/z']])
        print(pca.explained_variance_ratio_)
        df['PCA_Component 502 m/z'] = components


        # add columns for pct change
        def precent_change(col_name , periods):
            df[col_name + " pct change " + str(periods) + 'days'] = df[col_name].pct_change(periods=periods) * 100
            return df[col_name + " pct change " + str(periods) + 'days']

        for i in range (10):
            precent_change('EM Volts', i+1)
        for i in range (10):
            precent_change('502_to_69_ratio', i+1)

        val = pd.concat([val,df])
val.fillna(0, inplace=True) # fill all NaN values with 0
val

# Perform one-hot encoding
# one_hot_encoded = pd.get_dummies(val['System'], prefix='System')
# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# val = pd.concat([val, one_hot_encoded], axis=1)
#drop System column
val.drop(['System'], axis=1, inplace=True)
val.drop(['ID'], axis=1, inplace=True)
val

KeyError: "['Filament'] not found in axis"

In [ ]:
X_train = df1.drop(['Condition'], axis=1)
y_train = df1['Condition']
X_val = val.drop(['Condition'], axis=1)
y_val = val['Condition']


In [ ]:
X_val.shape, X_train.shape

((61, 39), (177, 39))

In [ ]:
X_train.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [ ]:
X_val.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [ ]:
X_train.columns

Index(['69 m/z', '70 m/z', '219 m/z', '220 m/z', '502 m/z', '503 m/z',
       'Amu gain', 'Amu offset', 'EM Volts', 'Ent Lens', 'Ion Focus',
       'Repeller', 'Ent Lens offset', 'Width219', '502_to_69_ratio',
       'total_hour_diff', 'PCA_Component 69 m/z', 'PCA_Component 219 m/z',
       'PCA_Component 502 m/z', 'EM Volts pct change 1days',
       'EM Volts pct change 2days', 'EM Volts pct change 3days',
       'EM Volts pct change 4days', 'EM Volts pct change 5days',
       'EM Volts pct change 6days', 'EM Volts pct change 7days',
       'EM Volts pct change 8days', 'EM Volts pct change 9days',
       'EM Volts pct change 10days', '502_to_69_ratio pct change 1days',
       '502_to_69_ratio pct change 2days', '502_to_69_ratio pct change 3days',
       '502_to_69_ratio pct change 4days', '502_to_69_ratio pct change 5days',
       '502_to_69_ratio pct change 6days', '502_to_69_ratio pct change 7days',
       '502_to_69_ratio pct change 8days', '502_to_69_ratio pct change 9days',
     

In [ ]:
# apply standard scalar
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [ ]:
# # apply over sampling
# from imblearn.over_sampling import SMOTE
# from collections import Counter

# sm = SMOTE(random_state=42)
# print('Original dataset shape %s' % Counter(y_train))
# X_train, y_train = sm.fit_resample(X_train, y_train)
# print('Resampled dataset shape %s' % Counter(y_train))


In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

rf = RandomForestClassifier(n_estimators = 5000)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("precision_score:", precision_score(y_val, y_pred))
print("recall_score:", recall_score(y_val, y_pred))
print("f1_score:", f1_score(y_val, y_pred))
print('roc_auc_score', roc_auc_score(y_val, y_pred))


Accuracy: 0.9672131147540983
precision_score: 0.0
recall_score: 0.0
f1_score: 0.0
roc_auc_score 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_val, y_pred

(0     0
 1     0
 2     0
 3     0
 4     0
      ..
 56    0
 57    0
 58    0
 59    0
 60    1
 Name: Condition, Length: 61, dtype: int64,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
# svc
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("precision_score:", precision_score(y_val, y_pred))
print("recall_score:", recall_score(y_val, y_pred))
print("f1_score:", f1_score(y_val, y_pred))
print('roc_auc_score', roc_auc_score(y_val, y_pred))


Accuracy: 0.9672131147540983
precision_score: 0.0
recall_score: 0.0
f1_score: 0.0
roc_auc_score 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_val, y_pred

(0     0
 1     0
 2     0
 3     0
 4     0
      ..
 56    0
 57    0
 58    0
 59    0
 60    1
 Name: Condition, Length: 61, dtype: int64,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential


In [ ]:
X_train.shape[1]

39

In [ ]:
epochs = 100
neurons = 32
dropout = 0.3
learning_rate = 0.0001
batch_size = 16
shapesize = X_train.shape[1]

model = tf.keras.Sequential([
        tf.keras.layers.Dense(neurons, activation='relu', input_shape=(shapesize,)),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(neurons, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(neurons, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),

        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam (learning_rate), loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

    history = model.fit(X_train, y_train,
                        epochs=epochs, batch_size=batch_size,
                        validation_data=( X_val, y_val ))

In [43]:
# def train_tf_model(X_train, y_train, X_val, y_val, neurons, dropout_rate, epochs, learning_rate=0.01, batch_size = 32, shapesize = X_train.shape[1]):


#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(neurons, activation='relu', input_shape=(shapesize,)),
#         tf.keras.layers.Dropout(dropout_rate),

#         tf.keras.layers.Dense(neurons, activation='relu'),
#         tf.keras.layers.Dropout(dropout_rate),

#         tf.keras.layers.Dense(neurons, activation='relu'),
#         tf.keras.layers.Dropout(dropout_rate),

#         tf.keras.layers.Dense(1, activation='sigmoid')
#     ])

#     model.compile(optimizer=tf.keras.optimizers.Adam (learning_rate), loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

#     history = model.fit(X_train, y_train,
#                         epochs=epochs, batch_size=batch_size,
#                         validation_data=( X_val, y_val ))

#     return history, model

In [1]:
# epochs = 100
# neurons = [16,32,64]
# dropout = [0,0.3,0.5]
# learning_rate = [0.01, 0.001, 0.0001]
# batch_size = [16, 32]
# df_DNN_result = pd.DataFrame()

# result_df = pd.DataFrame(columns=['neurons', 'dropout', 'learning_rate', 'batch_size', 'accuracy', 'val_accuracy', 'auc', 'val_auc'])

# for n in neurons:
#     for d in dropout:
#         for l in learning_rate:
#             for b in batch_size:
#                 history, model = train_tf_model(X_train, y_train, X_val, y_val, neurons=n, dropout_rate=d, epochs=epochs, learning_rate=l, batch_size=b)
#                 df_DNN_result = df_DNN_result.append(pd.DataFrame(history.history).assign(neurons=n, dropout=d, learning_rate=l, batch_size=b))
#                 print(f'neurons: {n}, dropout: {d}, learning_rate: {l}, batch_size: {b}')
#                 print(f'accuracy: {history.history["accuracy"][-1]}, val_accuracy: {history.history["val_accuracy"][-1]}')
#                 print(f'auc: {history.history["auc"][-1]}, val_auc: {history.history["val_auc"][-1]}')
#                 print(f'precision: {history.history["precision"][-1]}, val_precision: {history.history["val_precision"][-1]}')
#                 print(f'recall: {history.history["recall"][-1]}, val_recall: {history.history["val_recall"][-1]}')


#                 result = pd.DataFrame(history.history)

#                 result['neurons'] = n
#                 result['dropout'] = d
#                 result['learning_rate'] = l
#                 result['batch_size'] = b
#                 result_df = pd.concat([result_df, result], axis=0)
#                 print('')

In [ ]:
# def my_f1():
# 2 * (precision * recall) / (precision + recall)

In [ ]:
# result_df.to_csv('Result/time10 34 system.csv')